### Imports

In [83]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import subjects
from neuropy.utils.signal_process import filter_sig
from neuropy import plotting
from neuropy.analyses import Pf1D

### Example ripple trace

In [ ]:
sess = subjects.sd.ratSday3[0]

In [ ]:
# plt.style.use('dark_background')
figure = plotting.Fig()
fig, gs = figure.draw(grid=(3, 1),wspace=0.5)
maze = sess.paradigm['maze'].flatten()
rpls = sess.pbe.time_slice(maze[0],maze[1])
channels = sess.probegroup.get_connected_channels()

rpl_indx = [829] # visually identified with random selection runs
print(rpl_indx)
cmap = 'gist_rainbow'
for i,indx in enumerate(rpl_indx):
    ax = plt.subplot(gs[0])
    rpl = rpls[indx].flatten()
    rpl_sig = sess.eegfile.get_signal(
        channels[8][9], t_start=rpl[0]-0.20, t_stop=rpl[1]+0.2
    )
    plotting.plot_signal_traces(rpl_sig, pad=1,ax=ax,lw=2,color='b')
    ax.set_yticks([])

# figure.savefig(subjects.figpath_sd/'example_ripple')

### Example ripples and replay raster plot below it

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import subjects
from neuropy import plotting
from neuropy.analyses import Pf1D
sess = subjects.nsd.ratVday3[0]

In [ ]:
%matplotlib widget
# plt.style.use('dark_background')
figure = plotting.Fig()
fig, gs = figure.draw(grid=(3, 5),wspace=0.5)
maze = sess.paradigm['maze'].flatten()
rpls = sess.pbe.time_slice(maze[0],maze[1])
channels = sess.probegroup.get_connected_channels()
pyr_neurons = sess.neurons.get_neuron_type('pyr')
pos = sess.lin_maze
# pos.t_start = pos.t_start
run = sess.run['forward']
pf = Pf1D(neurons=pyr_neurons,position=pos,speed_thresh=10,sigma=4,grid_bin=3,frate_thresh=1,epochs=sess.run)
pf_neurons =pyr_neurons.get_by_id(pf.neuron_ids) 

tc = pf.tuning_curves
sort_ind = np.argsort(np.argmax(tc,axis=1))
pf_neurons = pf_neurons[sort_ind]

# rpl_indx = np.random.choice(len(rpls),4,replace=False)
rpl_indx = [356,712,426,829] # visually identified with random selection runs
print(rpl_indx)
cmap = 'gist_rainbow'
for i,indx in enumerate(rpl_indx):
    ax = plt.subplot(gs[0,i+1])
    rpl = rpls[indx].flatten()
    rpl_sig = sess.eegfile.get_signal(
        channels[8].astype(int), t_start=rpl[0]-0.10, t_stop=rpl[1]+0.1
    )
    plotting.plot_signal_traces(rpl_sig, pad=1,ax=ax,lw=0.5,color='k')
    if i>0:
        ax.set_yticks([])

    neurons = pf_neurons.time_slice(rpl[0],rpl[1])
    ax_raster = plt.subplot(gs[1,i+1])
    ax_raster = plotting.plot_raster(neurons,ax=ax_raster,color=cmap,markersize=4)
 
    ax_raster.axis('off')

ax_pf = plt.subplot(gs[1,0])
plotting.plot_ratemap(pf,ax=ax_pf,normalize_tuning_curve=True,cmap=cmap)
ax_pf.set_yticks([])
# figure.savefig(subjects.figpath_sd/'example_ripples_')

### Ripple traces multiple shanks

In [ ]:
import numpy as np
import subjects
import matplotlib.pyplot as plt
from neuropy.plotting import plot_signal_traces

sess = subjects.tn.ratKday3[0]
t1 = 349*60+56+0.9
# t1 = 350*60+0.2
t2 = t1+2
fig,ax = plt.subplots(5,1)
cmaps = ['r','#30f83a','#f830ee','#1eb1f6','#f6ee1e']
for i in range(5):
    channels = sess.recinfo.channel_groups[i+1].astype('int')
    signal = sess.eegfile.get_signal(channels,t1,t2)
    plot_signal_traces(signal,ax=ax[i],pad=0.2,color=cmaps[i],lw=0.5)
    ax[i].axis('off')
    # ax[i].set_facecolor('k')

fig.savefig(subjects.figpath_sd/'rpl_traces_multi_shanks.pdf')

### A sample run with CA1 cells, LFP, Actual position and decoded position
- something similar to Fig1C of Tang2021

In [ ]:
# sess = subjects.sd.ratVday2[0]
sess = subjects.nsd.ratUday2[0]
# sess = subjects.nsd.ratNday2[0]
maze = sess.paradigm["maze"].flatten()
maze_pos = sess.maze
maze_run = sess.maze_run
neurons = sess.neurons.get_neuron_type(['pyr','mua'])
good_cells = neurons.firing_rate<10
neurons = neurons[good_cells]
pf = Pf1D(
    neurons=neurons,
    position=maze_pos,
    sigma=4,
    grid_bin=2,
    # epochs=maze_run["down"],
    frate_thresh=0.02,
)



In [ ]:
from neuropy.analyses.decoders import Decode1d
pf_neurons = neurons.get_by_id(pf.neuron_ids)
decode = Decode1d(neurons=pf_neurons,ratemap=pf,bin_size=0.2)

In [ ]:
_,ax = plt.subplots()
# ax.imshow(decode.posterior,aspect='auto',cmap='binary',vmax=0.1)
ax.plot(decode.decoded_position,'.')
# ax.plot(,'.')

In [ ]:
# ---- Choose Period-----
# RatVDay2SD
# run_epoch = [12721, 12]
# pbe = [12722.5, 12723]


# RatUDay2NSD
# run_epoch = [11419, 11430.5]
# pbe = [11420.22, 11420.5]
# run_epoch = [11162, 11172]
# pbe = [11165.5, 11165.79]
run_epoch = [10744, 10760]
# pbe = [10758.1, 10758.55]
pbe = [10753.61, 10753.87]

# RatVDay2SD
# run_epoch = [maze[0] + 1060, maze[0] + 1070]
# pbe = [maze[0] + 1060 + 4.5, maze[0] + 1060 + 4.7]

# run_epoch = maze+5
# pbe = [maze[0]+100,maze[0]+100.5]

neurons_sorted = sess.neurons[pf.get_sort_order("neuron_id")].time_slice(*run_epoch)
pos_epoch = maze_pos.time_slice(*run_epoch)

cmap = "rainbow"
raster_kw = dict(markersize=4, markeredgewidth=2)
fig = plotting.Fig(grid=(16, 6))

# ---- Running epoch --------

# Raw LFP
sig_ax = fig.subplot(fig.gs[0, 1:6])
signal = sess.eegfile.get_signal(85, *run_epoch)
sig_ax.plot(signal.time, signal.traces[0], "k", lw=0.2)
sig_ax.set_xlim(run_epoch)
sig_ax.axis("off")

# Ripple band LFP
rpl_signal = filter_sig.bandpass(signal, lf=125, hf=250)
rpl_ax = fig.subplot(fig.gs[1, 1:6], sharex=sig_ax)
rpl_ax.plot(signal.time, rpl_signal.traces[0], "#4a77bae8", lw=0.2)
rpl_ax.set_xlim(run_epoch)
rpl_ax.axis("off")


# Raster plot
ax = fig.subplot(fig.gs[2:6, 1:6], sharex=sig_ax)
plotting.plot_raster(neurons_sorted, color=cmap, ax=ax, **raster_kw)

# Position plot
ax2 = ax.twinx()
ax2.plot(pos_epoch.time, pos_epoch.x, color="#a3a3a3", alpha=0.7, lw=3)
ax2.set_yticks([])
xticks = np.arange(run_epoch[0], run_epoch[1], 5)
# ax.set_xticks(xticks, xticks - xticks[0])
ax.tick_params(rotation=0)
ax.set_yticks([1, len(neurons_sorted)])

replay_ax = fig.subplot(fig.gs[2:6, 0])
plotting.plot_raster(
    neurons_sorted.time_slice(*pbe), ax=replay_ax, color=cmap, **raster_kw
)
replay_ax.axis("off")

sig_ax = fig.subplot(fig.gs[0, 0], sharex=replay_ax)
pbe_signal = signal.time_slice(None, *pbe)
sig_ax.plot(pbe_signal.time, pbe_signal.traces[0], "k", lw=0.5)
sig_ax.axis("off")


rpl_ax = fig.subplot(fig.gs[1, 0], sharex=replay_ax)
replay_sig = rpl_signal.time_slice(None, *pbe)
rpl_ax.plot(
    replay_sig.time, rpl_signal.time_slice(None, *pbe).traces[0], "#4a77bae8", lw=0.5
)
# rpl_ax.set_xlim(run_epoch)
rpl_ax.axis("off")


# All position plot 

ax = fig.subplot(fig.gs[6:9,2:4])
allpos = sess.position.time_slice(*maze)
run_pos = sess.position.time_slice(*run_epoch)
ax.plot(allpos.x,allpos.y,color='#d3d4d5')
ax.plot(run_pos.x,run_pos.y,color='r')
ax.axis('off')



In [ ]:
fig.savefig(subjects.figpath_sd/'example_run')

In [ ]:
_,ax = plt.subplots()

plotting.plot_ratemap(pf,normalize_tuning_curve=True,ax=ax)

### Ripple autocorrelation

In [ ]:
autocorr = subjects.GroupData().ripple_autocorr

fig = plotting.Fig(grid=(7, 3),fontsize=7)
ax = fig.subplot(fig.gs[0])
sns.lineplot(
    data=autocorr,
    x="lag_time",
    y="auto_corr",
    hue="grp",
    ci=None,
    palette=subjects.colors_sd(),
    hue_order=['NSD','SD']
)
ax.legend("", frameon=False)
fig.savefig(subjects.figpath_sd/'ripple_autocorr')

### Schematic of sharp wave amplitude method
- for use in sd_figure2

In [ ]:
from neuropy.utils.signal_process import filter_sig

sess = subjects.nsd.ratUday2[0]
sw_amp = sess.ripple.to_dataframe()['sharp_wave_amp']
# indx = np.random.choice(np.where(sw_amp>3)[0])
indx=17766
period = sess.ripple[indx].flatten()
period = [period[0]-0.1,period[1]+0.1]
channels = sess.recinfo.channel_groups[6].astype('int')[0::5]
signal = sess.eegfile.get_signal(channels,*period)
filt_signal = filter_sig.bandpass(signal,lf=2,hf=30)

# _,ax = plt.subplots()
fig = plotting.Fig(grid=(11,20))
ax = fig.subplot(fig.gs[0,:2])
plotting.plot_signal_traces(signal,ax=ax,pad=0,color='#949494',lw=0.3)
plotting.plot_signal_traces(filt_signal,ax=ax,color='#5d22a0',pad=0,lw=0.5)
ax.axis('off')

fig.savefig(subjects.figpath_sd/'sw_amp_example')

### EV for 2 animals 

In [ ]:
from neuropy.core import Epoch

errorbar_kw = dict(alpha=0.5, linewidth=0, ec=None)

grpdata = subjects.GroupData()
ev_df = grpdata.ev_pooled

ev_states = grpdata.ev_brainstates
indx = np.isin(ev_states.name, ["RatN", "RatS"])
ev_states = ev_states[indx]

ev_colors = subjects.colors_sd(0.8) 

plt.close()
fig = plotting.Fig(grid=(9, 4))
subfig = fig.add_subfigure(fig.gs[:2, :2])
axs = subfig.subplots(
    4, 2, sharey="row", sharex="col", gridspec_kw=dict(height_ratios=[1, 4, 1, 4])
)

for row, rat in enumerate(['RatN','RatS']):
    for col,grp in enumerate(['NSD','SD']):
        color = ev_colors[0] if grp == "NSD" else ev_colors[1]

        ax = axs[2*row,col]
        epoch = Epoch(ev_states[(ev_states.name==rat)&(ev_states.grp==grp)])
        plotting.plot_hypnogram(epoch, ax=ax, collapsed=False)


        ax = axs[2*row+1,col]

        df = ev_df[(ev_df.name==rat)&(ev_df.grp==grp)]
        t, ev, ev_err, rev, rev_err = (
            df["Zt"],
            df["ev"],
            df["ev_std"],
            df["rev"],
            df["rev_std"],
        )

        ax.fill_between(t, rev - rev_err, rev + rev_err, color="#33d675", **errorbar_kw)
        ax.plot(t, rev, color="#33d675")
        ax.fill_between(t, ev - ev_err, ev + ev_err, color=color, **errorbar_kw)
        ax.plot(t, ev, color=color)

fig.savefig(subjects.figpath_sd/'example_ev')